In [12]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [13]:
!unzip "/content/drive/MyDrive/data_trimmed_clean.zip" -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/data_trimmed/Train/Shooting/Shooting042_x264_1030.png  
 extracting: /content/data_trimmed/Train/Shooting/Shooting020_x264_2680.png  
  inflating: /content/data_trimmed/Train/Shooting/Shooting029_x264_1260.png  
  inflating: /content/data_trimmed/Train/Shooting/Shooting009_x264_2690.png  
 extracting: /content/data_trimmed/Train/Shooting/Shooting014_x264_2740.png  
  inflating: /content/data_trimmed/Train/Shooting/Shooting006_x264_4510.png  
  inflating: /content/data_trimmed/Train/Shooting/Shooting006_x264_11010.png  
  inflating: /content/data_trimmed/Train/Shooting/Shooting006_x264_9020.png  
  inflating: /content/data_trimmed/Train/Shooting/Shooting005_x264_1860.png  
 extracting: /content/data_trimmed/Train/Shooting/Shooting052_x264_4560.png  
  inflating: /content/data_trimmed/Train/Shooting/Shooting009_x264_130.png  
 extracting: /content/data_trimmed/Train/Shooting/Shooting027_x264_140.png  
  inflating: /

In [3]:
# ONLY USED FOR TESTING AND DEBUGGING - for final model we will use the whole dataset

import os
import shutil

def create_debug_subset_sequential(source_dir, dest_dir, train_limit=4000, test_limit=1500):
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
    os.makedirs(dest_dir, exist_ok=True)

    for split, limit in [("Train", train_limit), ("Test", test_limit)]:
        src_split_path = os.path.join(source_dir, split)
        dst_split_path = os.path.join(dest_dir, split)
        os.makedirs(dst_split_path, exist_ok=True)

        for class_name in os.listdir(src_split_path):
            class_src = os.path.join(src_split_path, class_name)
            class_dst = os.path.join(dst_split_path, class_name)
            os.makedirs(class_dst, exist_ok=True)

            valid_images = sorted([f for f in os.listdir(class_src) if f.endswith(".png") and not f.startswith("._")])
            selected_images = valid_images[:limit]

            for img in selected_images:
                shutil.copy(os.path.join(class_src, img), os.path.join(class_dst, img))

create_debug_subset_sequential("/content/data_trimmed", "/content/data_trimmed_debug", train_limit=4000, test_limit=1500)

# paths
train_dir = "/content/data_trimmed_debug/Train"
test_dir = "/content/data_trimmed_debug/Test"

In [4]:
import os
import shutil
from collections import defaultdict

def extract_video_id(filename):
    """
    Extracts the video ID by removing the frame suffix (assumes last underscore + digits is the frame number).
    For example:
        Normal_Videos_003_x264_0.png → Normal_Videos_003_x264
        Assault_001_frame_010.png → Assault_001
    """
    parts = filename.rsplit("_", 1)
    return parts[0] if len(parts) == 2 else filename.split("_frame")[0]

def restructure_dataset(src_dir, dst_dir):
    os.makedirs(dst_dir, exist_ok=True)

    for class_name in os.listdir(src_dir):
        class_path = os.path.join(src_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        print(f"Processing class: {class_name}")
        video_frame_dict = defaultdict(list)

        for fname in os.listdir(class_path):
            if not fname.endswith('.png'):
                continue

            video_id = extract_video_id(fname)
            video_frame_dict[video_id].append(fname)

        for video_id, frames in video_frame_dict.items():
            video_folder_path = os.path.join(dst_dir, class_name, video_id)
            os.makedirs(video_folder_path, exist_ok=True)

            for frame in frames:
                src = os.path.join(class_path, frame)
                dst = os.path.join(video_folder_path, frame)
                shutil.copy2(src, dst)

    print(f"Done restructuring: {src_dir} → {dst_dir}\n")

# Paths
train_dir = "/content/data_trimmed_debug/Train"
test_dir = "/content/data_trimmed_debug/Test"

train_dst = "/content/data_trimmed_debug_restructured/Train"
test_dst = "/content/data_trimmed_debug_restructured/Test"

# Run restructuring
restructure_dataset(train_dir, train_dst)
restructure_dataset(test_dir, test_dst)

Processing class: Arson
Processing class: Arrest
Processing class: Shooting
Processing class: Burglary
Processing class: Explosion
Processing class: NormalVideos
Processing class: Assault
Processing class: Fighting
Done restructuring: /content/data_trimmed_debug/Train → /content/data_trimmed_debug_restructured/Train

Processing class: Arson
Processing class: Arrest
Processing class: Shooting
Processing class: Burglary
Processing class: Explosion
Processing class: NormalVideos
Processing class: Assault
Processing class: Fighting
Done restructuring: /content/data_trimmed_debug/Test → /content/data_trimmed_debug_restructured/Test



In [5]:
IMG_HEIGHT = 64
IMG_WIDTH = 64
SEQUENCE_LENGTH = 16  # Pad/truncate each video to this many frames
BATCH_SIZE = 8
CLASS_NAMES = ['Arrest','Arson','Assault','Burglary','Explosion','Fighting','NormalVideos','Shooting']
NUM_CLASSES = len(CLASS_NAMES)

In [6]:
import tensorflow as tf
import numpy as np
import os
from glob import glob
from tensorflow.keras.utils import to_categorical
from PIL import Image

def load_video_frames(video_dir, sequence_length, img_size):
    # Get sorted list of frame paths
    frame_paths = sorted(glob(os.path.join(video_dir, "*.png")))

    frames = []
    for path in frame_paths[:sequence_length]:
        img = Image.open(path).resize(img_size)
        frame = np.array(img).astype("float32") / 255.0  # Normalize to [0, 1]
        frames.append(frame)

    # Pad with zeros if not enough frames
    while len(frames) < sequence_length:
        frames.append(np.zeros((img_size[1], img_size[0], 3), dtype="float32"))

    return np.stack(frames)

def get_video_paths_and_labels(base_dir, class_names):
    video_paths = []
    labels = []

    for class_index, class_name in enumerate(class_names):
        class_path = os.path.join(base_dir, class_name)
        for video_folder in os.listdir(class_path):
            video_path = os.path.join(class_path, video_folder)
            if os.path.isdir(video_path):
                video_paths.append(video_path)
                labels.append(class_index)

    return video_paths, labels

def build_video_dataset(base_dir, sequence_length, img_size, batch_size, class_names, shuffle=True):
    video_paths, labels = get_video_paths_and_labels(base_dir, class_names)

    def generator():
        for video_path, label in zip(video_paths, labels):
            frames = load_video_frames(video_path, sequence_length, img_size)
            yield frames, to_categorical(label, num_classes=len(class_names))

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(sequence_length, img_size[1], img_size[0], 3), dtype=tf.float32),
            tf.TensorSpec(shape=(len(class_names),), dtype=tf.float32)
        )
    )

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(video_paths))

    return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [7]:
train_seq_dir = "/content/data_trimmed_debug_restructured/Train"
test_seq_dir = "/content/data_trimmed_debug_restructured/Test"

train_dataset = build_video_dataset(
    base_dir=train_seq_dir,
    sequence_length=SEQUENCE_LENGTH,
    img_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_names=CLASS_NAMES,
    shuffle=True
)

test_dataset = build_video_dataset(
    base_dir=test_seq_dir,
    sequence_length=SEQUENCE_LENGTH,
    img_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_names=CLASS_NAMES,
    shuffle=False
)

In [8]:
for x, y in train_dataset.take(1):
    print(x.shape)  # (8, 16, 64, 64, 3)
    print(y.shape)  # (8, 8)

(8, 16, 64, 64, 3)
(8, 8)


## DINOv2 + LSTM (using PyTorch)

In [11]:
## run just once on colab!
!pip install transformers datasets timm

In [12]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.transforms import Compose, Resize, ToTensor
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
from glob import glob
from sklearn.preprocessing import LabelEncoder
import random

# Constants
SEQUENCE_LENGTH = 16
IMG_SIZE = 224
NUM_CLASSES = 8
BATCH_SIZE = 4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
CLASS_NAMES = ['Arrest','Arson','Assault','Burglary','Explosion','Fighting','NormalVideos','Shooting']

# Label encoder
label_encoder = LabelEncoder()
label_encoder.fit(CLASS_NAMES)

LabelEncoder()

In [22]:
from torchvision import transforms


In [23]:
class VideoDataset(Dataset):
    def __init__(self, base_dir, sequence_length=16, image_size=224, augment=False):
        self.sequence_length = sequence_length
        self.image_size = image_size
        self.samples = []
        self.processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
        self.augment = augment

        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(image_size, scale=(0.7, 1.0)),              # more aggressive crop
            transforms.RandomHorizontalFlip(p=0.5),                                  # flip 50% of the time
            transforms.RandomApply([
                transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4)
            ], p=0.8),
            transforms.RandomRotation(degrees=10),                                   # rotate frames a bit
            transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.95, 1.05)),  # slight shift/zoom
            transforms.RandomApply([
                transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))
            ], p=0.3)
        ])

        for class_name in os.listdir(base_dir):
            class_path = os.path.join(base_dir, class_name)
            if not os.path.isdir(class_path): continue
            for video_folder in os.listdir(class_path):
                video_path = os.path.join(class_path, video_folder)
                if os.path.isdir(video_path):
                    frame_paths = sorted(glob(os.path.join(video_path, '*.png')))
                    if frame_paths:
                        self.samples.append((frame_paths, label_encoder.transform([class_name])[0]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        frame_paths, label = self.samples[idx]
        selected = frame_paths[:self.sequence_length]

        while len(selected) < self.sequence_length:
            selected.append(selected[-1])

        images = []
        for path in selected:
            img = Image.open(path).convert("RGB").resize((self.image_size, self.image_size))
            if self.augment:
                img = self.transform(img)
            images.append(img)

        processed = self.processor(images=images, return_tensors="pt")
        pixel_values = processed['pixel_values'].squeeze(0)
        label = torch.tensor(label, dtype=torch.long)
        return pixel_values, label

# Paths
train_dir = "/content/data_trimmed_debug_restructured/Train"
test_dir = "/content/data_trimmed_debug_restructured/Test"

train_dataset = VideoDataset(train_dir, sequence_length=SEQUENCE_LENGTH, image_size=IMG_SIZE, augment=True)
test_dataset = VideoDataset(test_dir, sequence_length=SEQUENCE_LENGTH, image_size=IMG_SIZE, augment=False)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


In [24]:
class DINOv2_LSTM(nn.Module):
    def __init__(self, model_name='facebook/dinov2-base', hidden_dim=128, num_classes=NUM_CLASSES):
        super().__init__()
        self.vit = AutoModel.from_pretrained(model_name)
        self.vit.eval()
        for param in self.vit.parameters():
            param.requires_grad = False

        self.lstm = nn.LSTM(input_size=768, hidden_size=hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):  # x: (B, T, 3, 224, 224)
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)

        with torch.no_grad():
            vit_out = self.vit(pixel_values=x).last_hidden_state[:, 0]  # CLS token

        x_seq = vit_out.view(B, T, -1)  # (B, T, 768)
        x_seq, _ = self.lstm(x_seq)
        x_seq = self.dropout(x_seq[:, -1, :])  # last time step
        return self.fc(x_seq)

# Model
model = DINOv2_LSTM().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [25]:
from sklearn.metrics import accuracy_score, roc_auc_score
import torch.nn.functional as F

def evaluate(model, loader, criterion, split='Test'):
    model.eval()
    total_loss = 0.0
    total_preds = []
    total_probs = []
    total_labels = []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            logits = model(x)
            probs = F.softmax(logits, dim=1)
            loss = criterion(logits, y)

            total_loss += loss.item()
            total_probs.append(probs.cpu())
            total_preds.append(torch.argmax(probs, dim=1).cpu())
            total_labels.append(y.cpu())

    y_true = torch.cat(total_labels).numpy()
    y_pred = torch.cat(total_preds).numpy()
    y_probs = torch.cat(total_probs).numpy()

    acc = accuracy_score(y_true, y_pred)

    try:
        auc = roc_auc_score(y_true, y_probs, multi_class='ovr', average='macro')
    except ValueError:
        auc = 0.0

    avg_loss = total_loss / len(loader)
    print(f"{split} Loss: {avg_loss:.4f} | Accuracy: {acc:.4f} | AUC: {auc:.4f}")

    return avg_loss  # ✅ Ensure test loss is returned


## DINOv2 + Transformer Encoder (using PyTorch)

In [26]:
import torch
import torch.nn as nn
from transformers import AutoModel

class DINOv2_Transformer(nn.Module):
    def __init__(self, model_name='facebook/dinov2-base', hidden_dim=768, num_classes=8, num_heads=4, ff_dim=1024, dropout=0.1):
        super().__init__()

        # DINOv2 backbone
        self.vit = AutoModel.from_pretrained(model_name)
        self.vit.eval()
        for param in self.vit.parameters():
            param.requires_grad = False

        # Transformer Encoder block
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=1)

        # Final classifier
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):  # x: (B, T, 3, 224, 224)
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)

        with torch.no_grad():
            vit_out = self.vit(pixel_values=x).last_hidden_state[:, 0]  # CLS token

        x_seq = vit_out.view(B, T, -1)  # (B, T, 768)
        x_encoded = self.transformer_encoder(x_seq)  # (B, T, 768)
        x_pooled = x_encoded.mean(dim=1)  # average over time
        return self.fc(self.dropout(x_pooled))

model = DINOv2_Transformer().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [27]:
for epoch in range(8):
    model.train()
    running_loss = 0.0
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"\nEpoch {epoch+1}, Train Loss: {avg_train_loss:.4f}")
    evaluate(model, train_loader, criterion, split='Train')
    evaluate(model, test_loader, criterion, split='Test')


Epoch 1, Train Loss: 2.1974
Train Loss: 1.4303 | Accuracy: 0.4857 | AUC: 0.8873
Test Loss: 2.1561 | Accuracy: 0.2727 | AUC: 0.5996

Epoch 2, Train Loss: 1.3660
Train Loss: 0.9400 | Accuracy: 0.7333 | AUC: 0.9793
Test Loss: 2.0993 | Accuracy: 0.2273 | AUC: 0.6044

Epoch 3, Train Loss: 0.8875
Train Loss: 0.6089 | Accuracy: 0.8667 | AUC: 0.9969
Test Loss: 2.1839 | Accuracy: 0.2500 | AUC: 0.6136

Epoch 4, Train Loss: 0.5818
Train Loss: 0.3713 | Accuracy: 0.9714 | AUC: 0.9998
Test Loss: 2.2630 | Accuracy: 0.2273 | AUC: 0.6258

Epoch 5, Train Loss: 0.3693
Train Loss: 0.2002 | Accuracy: 1.0000 | AUC: 1.0000
Test Loss: 2.4337 | Accuracy: 0.2273 | AUC: 0.6176

Epoch 6, Train Loss: 0.1987
Train Loss: 0.1098 | Accuracy: 1.0000 | AUC: 1.0000
Test Loss: 2.4875 | Accuracy: 0.2045 | AUC: 0.6464

Epoch 7, Train Loss: 0.1119
Train Loss: 0.0622 | Accuracy: 1.0000 | AUC: 1.0000
Test Loss: 2.6020 | Accuracy: 0.2500 | AUC: 0.6424

Epoch 8, Train Loss: 0.0725
Train Loss: 0.0460 | Accuracy: 1.0000 | AUC: 1.

Fine tuning the model

In [28]:
import torch
import torch.nn as nn
from transformers import AutoModel

class DINOv2_Transformer(nn.Module):
    def __init__(self, model_name='facebook/dinov2-base', hidden_dim=768, num_classes=8, num_heads=4, ff_dim=1024, dropout=0.2):
        super().__init__()

        # Fine-Tuning 1: Load pretrained DINOv2 and make it trainable
        # Previously: self.vit.eval(); all parameters were frozen
        self.vit = AutoModel.from_pretrained(model_name)
        self.vit.train()
        for param in self.vit.parameters():
            param.requires_grad = True  # Fine-Tuning 1 applied here

        # Fine-Tuning 2: Add dropout to the Transformer encoder for regularization
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,  # Fine-Tuning 2
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=1)

        # Standard classification head
        self.norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):  # x: (batch, time, 3, 224, 224)
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)

        vit_out = self.vit(pixel_values=x).last_hidden_state[:, 0]  # Get CLS tokens

        x_seq = vit_out.view(B, T, -1)  # Reshape to sequence
        x_encoded = self.transformer_encoder(x_seq)
        x_pooled = self.norm(x_encoded.mean(dim=1))
        return self.fc(self.dropout(x_pooled))


In [29]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Fine-Tuning 3: Label smoothing to reduce overconfidence in predictions
# Previously: standard CrossEntropyLoss
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Fine-Tuning 3

# Separate vit and head parameters
vit_params = []
head_params = []

for name, param in model.named_parameters():
    if "vit" in name:
        vit_params.append(param)
    else:
        head_params.append(param)

# Fine-Tuning 4: Use AdamW optimizer with different learning rates
# Fine-Tuning 5: Add weight decay for L2 regularization
# Previously: Single LR, Adam optimizer, no weight decay
optimizer = AdamW(
    [
        {"params": vit_params, "lr": 1e-5},    # Fine-Tuning 4
        {"params": head_params, "lr": 1e-4}     # Fine-Tuning 4
    ],
    weight_decay=0.01  # Fine-Tuning 5
)

# Fine-Tuning 6: Learning rate scheduler to keep learning if loss plateaus
# Previously: no scheduler used
scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=2,
    verbose=True,
    min_lr=1e-6
)  # Fine-Tuning 6


fine tune 2

In [30]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn

# Create the model
model = DINOv2_Transformer().to(DEVICE)

# Loss function with label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Split parameters into groups: vit (partially trainable) and the head
vit_params = []
head_params = []

for name, param in model.named_parameters():
    if param.requires_grad:
        if "vit" in name:
            vit_params.append(param)
        else:
            head_params.append(param)

# Optimizer with different learning rates for each part
optimizer = AdamW([
    {"params": vit_params, "lr": 3e-6},
    {"params": head_params, "lr": 5e-5}
], weight_decay=0.01)

# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

# Best model tracking
best_test_loss = float('inf')
best_model_path = "best_model.pt"

# Training loop with 10 epochs
for epoch in range(10):
    model.train()
    running_loss = 0.0

    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"\nEpoch {epoch+1}, Train Loss: {avg_train_loss:.4f}")

    # Evaluation
    evaluate(model, train_loader, criterion, split='Train')
    test_loss = evaluate(model, test_loader, criterion, split='Test')

    # Save best model
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"Model saved at epoch {epoch+1}")

    # Update learning rate
    scheduler.step(avg_train_loss)



Epoch 1, Train Loss: 2.1320
Train Loss: 1.4287 | Accuracy: 0.5524 | AUC: 0.9213
Test Loss: 2.0783 | Accuracy: 0.2045 | AUC: 0.5742
Model saved at epoch 1

Epoch 2, Train Loss: 1.3112
Train Loss: 0.9468 | Accuracy: 0.9143 | AUC: 0.9959
Test Loss: 2.0584 | Accuracy: 0.2955 | AUC: 0.5930
Model saved at epoch 2

Epoch 3, Train Loss: 0.9661
Train Loss: 0.7049 | Accuracy: 0.9429 | AUC: 0.9994
Test Loss: 2.1218 | Accuracy: 0.2500 | AUC: 0.6349

Epoch 4, Train Loss: 0.7473
Train Loss: 0.5551 | Accuracy: 0.9905 | AUC: 1.0000
Test Loss: 2.1255 | Accuracy: 0.2727 | AUC: 0.6379

Epoch 5, Train Loss: 0.5814
Train Loss: 0.5400 | Accuracy: 1.0000 | AUC: 1.0000
Test Loss: 2.2593 | Accuracy: 0.3182 | AUC: 0.6196

Epoch 6, Train Loss: 0.5298
Train Loss: 0.5045 | Accuracy: 1.0000 | AUC: 1.0000
Test Loss: 2.2120 | Accuracy: 0.2273 | AUC: 0.6513

Epoch 7, Train Loss: 0.5243
Train Loss: 0.5183 | Accuracy: 1.0000 | AUC: 1.0000
Test Loss: 2.2064 | Accuracy: 0.2727 | AUC: 0.6426

Epoch 8, Train Loss: 0.5276
T